# Lightkurve-only periodogram notebook

This notebook uses **only the Lightkurve package's convenience API** (no direct Astropy BLS calls).
It:
- loads your CSV and forces the `Flux` column (second column),
- computes a BLS periodogram via `lc.to_periodogram(method='bls', ...)`,
- computes a Lomb-Scargle periodogram via Lightkurve if desired (also via `to_periodogram`),
- produces plots of periodograms and overlays the Lightkurve model (`pg.model`) on the light curve,
- folds & bins the light curve using Lightkurve helpers and overlays the folded model.

Defaults: period range 0.5–25 days; use BLS by default. Generated: 2025-09-18 07:54 UTC

---



In [ ]:
# Install required packages if needed (uncomment to run)
# !pip install lightkurve numpy pandas matplotlib
# Restart the kernel after installing if prompted.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightkurve as lk

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,4)


In [ ]:
# === EDIT THIS PATH IF YOUR FILE IS LOCATED ELSEWHERE ===
DATA_PATH = "tic437011608flattened-2min.csv"

df = pd.read_csv(DATA_PATH)
print('Columns found:', df.columns.tolist())
display(df.head())

# Detect time column (auto-detect based on 'time' or 'bjd')
time_col_candidates = [c for c in df.columns if 'time' in c.lower() or 'bjd' in c.lower()]
time_col = time_col_candidates[0] if time_col_candidates else df.columns[0]

# Force use of the 'Flux' column (2nd column) instead of Flattened Flux
if 'Flux' in df.columns:
    flux_col = 'Flux'
else:
    flux_col = df.columns[1]  # fallback to second column

time_rel = df[time_col].values.astype(float)   # user provided time is BJD-2457000 per header
time_bjd = time_rel + 2457000.0
flux = df[flux_col].values.astype(float)

print(f"Using time column: '{time_col}' (interpreted as BJD-2457000).")
print(f"Using flux column: '{flux_col}'. Number of points: {len(time_rel)}")

# Create a Lightkurve LightCurve object
lc = lk.LightCurve(time=time_rel, flux=flux)
print('LightCurve created. Summary:')
display(lc.head())


In [ ]:
# ===== BLS periodogram (Lightkurve convenience call) =====
min_period = 0.5
max_period = 25.0

print('Computing BLS periodogram via Lightkurve...')
# to_periodogram(method='bls') will use Lightkurve's BLS convenience wrapper
pg_bls = lc.to_periodogram(method='bls', minimum_period=min_period, maximum_period=max_period)

# Plot periodogram (period on x-axis)
ax = pg_bls.plot(view='period', normalize=False)
plt.title('BLS Periodogram (Lightkurve)')
plt.show()

# Report best period
try:
    best_period_bls = pg_bls.period_at_max_power
    print('BLS best period (period_at_max_power):', best_period_bls)
except Exception:
    # fallback to compute from frequency_at_max_power if needed
    try:
        best_period_bls = 1.0 / pg_bls.frequency_at_max_power
        print('BLS best period (from frequency_at_max_power):', best_period_bls)
    except Exception:
        best_period_bls = None
        print('Could not determine best BLS period from pg_bls object. Inspect pg_bls attributes.')

# If the periodogram object exposes transit_time, print it
if hasattr(pg_bls, 'transit_time'):
    try:
        print('BLS reported transit_time:', pg_bls.transit_time)
    except Exception:
        pass


In [ ]:
# ===== Lomb-Scargle periodogram via Lightkurve (optional) =====
do_lombscargle = True  # set to False to skip LS
if do_lombscargle:
    print('Computing Lomb-Scargle periodogram via Lightkurve...')
    pg_ls = lc.to_periodogram(method='lombscargle', minimum_period=min_period, maximum_period=max_period)
    ax = pg_ls.plot(view='period', normalize=False)
    plt.title('Lomb-Scargle Periodogram (Lightkurve)')
    plt.show()
    try:
        best_period_ls = pg_ls.period_at_max_power
        print('LS best period:', best_period_ls)
    except Exception:
        best_period_ls = None


In [ ]:
# ===== Overlay periodogram-derived model on the light curve (Lightkurve tutorial-style) =====
# Uses the model() method of the Lightkurve Periodogram object to produce a LightCurve model
# and overlays it on the original light curve.

# Choose which periodogram to use for model (prefer BLS)
pg = pg_bls if 'pg_bls' in globals() else (pg_ls if 'pg_ls' in globals() else None)
if pg is None:
    raise RuntimeError('No periodogram object found. Run the periodogram cells first.')

# Create the model LightCurve aligned with lc.time
# Try to pass period first (BLS Periodogram supports period argument). If that fails, pass frequency.
try:
    lc_model = pg.model(time=lc.time, period=pg.period_at_max_power)
except Exception:
    lc_model = pg.model(time=lc.time, frequency=pg.frequency_at_max_power)

# Plot full light curve and overlay model
ax = lc.plot(marker='.', markersize=2, alpha=0.6)
lc_model.plot(ax=ax, lw=3, ls='--', c='red', label='Periodogram model')
ax.set_title(f'Full light curve with Periodogram model (P = {pg.period_at_max_power})')
ax.legend()
plt.show()


In [ ]:
# ===== Fold, bin, and overlay folded model using Lightkurve helpers =====
# Determine best period and an epoch_time to fold around
pg_use = pg_bls if 'pg_bls' in globals() else (pg_ls if 'pg_ls' in globals() else None)
if pg_use is None:
    raise RuntimeError('No periodogram object available for folding.')

# Best period (attempt)
try:
    best_period = float(pg_use.period_at_max_power)
except Exception:
    best_period = float(1.0 / pg_use.frequency_at_max_power)

# Try to get transit_time from periodogram if available
epoch_time = None
if hasattr(pg_use, 'transit_time'):
    try:
        epoch_time = float(pg_use.transit_time)
    except Exception:
        epoch_time = None

# Fallback epoch: use first time value
if epoch_time is None:
    epoch_time = float(lc.time.value[0])

print('Folding at period =', best_period, 'using epoch_time =', epoch_time)

# Fold using Lightkurve's fold() method and then bin
lc_folded = lc.fold(period=best_period, epoch_time=epoch_time)
# Lightkurve LightCurve has a bin() method to bin by number of bins or time interval
lc_folded_binned = lc_folded.bin(bins=100)   # bin into 100 phase bins

# Fold the model to same epoch/time grid for overlay
try:
    model_folded = lc_model.fold(period=best_period, epoch_time=epoch_time)
except Exception:
    # If lc_model doesn't exist or folding fails, attempt to create a model on folded times
    model_folded = pg_use.model(time=lc_folded.time, period=best_period)

# Plot folded & binned
ax = lc_folded.plot(marker='.', alpha=0.2, label='data (folded)')
lc_folded_binned.plot(ax=ax, marker='o', label='binned median')
model_folded.plot(ax=ax, lw=2, ls='--', c='C1', label='folded model')
ax.set_xlim(0, best_period)  # Lightkurve default fold keeps time as phase-like; adjust as needed
ax.set_xlabel(f'Phase (days) folded at P={best_period:.6f} d')
ax.set_ylabel('Flux')
ax.set_title('Phase-folded & binned light curve with model (Lightkurve)')
ax.legend()
plt.show()


In [ ]:
# Optionally save results and model
results = {}
if 'best_period' in globals():
    results['best_period'] = float(best_period)
if 'pg_bls' in globals() and hasattr(pg_bls, 'period_at_max_power'):
    results['best_period_bls'] = float(pg_bls.period_at_max_power)
if 'pg_ls' in globals() and hasattr(pg_ls, 'period_at_max_power'):
    results['best_period_ls'] = float(pg_ls.period_at_max_power)

import json
with open('lightkurve_periodogram_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print('Saved results to lightkurve_periodogram_results.json:', results)


---
Notes:
- This notebook exclusively uses Lightkurve convenience methods (lc.to_periodogram, pg.model, lc.fold, lc.bin).
- It avoids any direct calls to Astropy's BoxLeastSquares/LombScargle in the notebook.
- Tweak `min_period`, `max_period`, and `lc_folded.bin(bins=...)` to control resolution and runtime.
---
